<a href="https://colab.research.google.com/github/gabrielrudan/RAG_Dialogue_OPN/blob/master/Gera%C3%A7%C3%A3o_de_Di%C3%A1logos_com_RAG_RPG_Ordem_Paranormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas** do Projeto

## - Instalações necessárias -

In [ ]:
!pip install -U langchain-openai
!pip install langchain
!pip install qdrant-client
!pip install langchain_community

## - Importando as bibliotecas -

### **ChatOpenAI** e chave da **API**

In [ ]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
chat_openai = ChatOpenAI(api_key=api_key, model='gpt-4-turbo-preview')

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

### **Pré-processamento** do texto do livro

In [ ]:
import re
from langchain.text_splitter import TextSplitter

In [ ]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')

### **Embeddings** e busca vetorial no **Vector DB**

In [ ]:
from langchain.schema import Document

In [ ]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

# Base de texto - **Livro de RPG** do ***Ordem Paranormal***

## - Trazendo o **Conjunto de Texto** -

In [ ]:
# Abrir o arquivo no modo de leitura
with open('livro_regras_rpg.txt', 'r', encoding='utf-8') as file:
    # Ler todo o conteúdo do arquivo e armazenar na variável 'text'
    text = file.read()

## - Separando o texto em **Seções** -

In [ ]:
class FlexibleTextSplitter(TextSplitter):
    def __init__(self, headers_to_split_on):
        self.headers_to_split_on = headers_to_split_on

    def split_text(self, text):
        # Junta os padrões em uma única expressão regular
        padrao_composto = '|'.join(f'({p})' for p in self.headers_to_split_on)
        # Divide o texto com base nos padrões
        partes = re.split(padrao_composto, text)
        # Remove entradas vazias e combina texto que possa estar dividido de forma inadequada
        # Filter out None values before applying .strip()
        seções = [parte.strip() for parte in partes if parte is not None and parte.strip()]
        return seções

# Ajuste os padrões conforme o texto
padrões_seção = [
    r'\bINTRODUÇÃO\b',
    r'\bCAPÍTULO \d+:\b',
    r'\bSUBSEÇÃO \d+:\b',
    r'\b[0-9]+\. [A-Za-zÀ-ÿ]+\b'  # Para capturar padrões de seções adicionais
]

# Cria uma instância do FlexibleTextSplitter
flexible_splitter = FlexibleTextSplitter(headers_to_split_on=padrões_seção)

# Divide o texto em seções com base nos padrões definidos
seções = flexible_splitter.split_text(text)

# Exibe as seções divididas para revisão
for i, secao in enumerate(seções):
    print(f"Seção {i+1}:\n{secao[:200]}...")  # Exibe os primeiros 200 caracteres de cada seção

# Se desejar a lista de seções
print("\nTotal de seções divididas:", len(seções))

Seção 1:
Ahistória de Ordem Paranormal começou do mesmo jeito que todas as suas missões vão começar: um grupo de amigos se reunindo emum fim de semana para se divertir juntos.Em 2019, eu tive uma ideia simples...
Seção 2:
318. Rua...
Seção 3:
Coronel Genuíno, 209 • Porto Alegre, rs 1ª edição: setembro de 2022 | ISBN: 978658863413-4 [email protected] www.jamboeditora.com.br L274o Lange, Rafael @jamboeditora Ordem paranormal RPG / Rafael Lan...
Seção 4:
1. Literatura...
Seção 5:
brasileira - Ficção científica. I. Svaldi, Guilherme Dei. II. Título. CDU 794:681.312
Essa é uma ação arriscada, pois Bianca pode ser vista I Pessoa: um ser humano, seja personagem, seja pelos guardas...
Seção 6:
16. Bianca...
Seção 7:
de uma entidade do Outro Lado, geralmente com a escolhe o mais alto deles,...
Seção 8:
16. Ela...
Seção 9:
é treinada em Fur- forma de um monstro terrível. Um zumbi de san- tividade, então soma +5 ao resultado, para um valor gue, um existido e uma degolificada são criaturas. final 

# **RAG** - Retrieval Augmented Generation

## - **Embeddings** -

In [ ]:
docs = []
for i, texto in enumerate(seções):
    seção_titulo = f"Seção {i+1}"  # Cria o título da seção com base no índice
    doc = Document(metadata={'source': seção_titulo}, page_content=texto)
    docs.append(doc)

In [ ]:
embeddings = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')

qdrant = Qdrant.from_documents(
    documents = docs,
    embedding = embeddings,
    location=":memory:",
    collection_name="rpg_ordem_paranormal"
)

## - Aumentando o **prompt** com o **Contexto do Livro** -

In [ ]:
def custom_prompt(query: str):
  results = qdrant.similarity_search(query, k=2)
  source_knowlegde = "\n".join([x.page_content for x in results])
  augmented_prompt = f"""Use o contexto abaixo para responder à pergunta.

  Contexto:
  {source_knowlegde}

  Pergunta:
  {query}"""
  return augmented_prompt

# Interação com o **modelo gpt-4.0 com RAG**

In [ ]:
def interacao_com_modelo():
    messages = [
        SystemMessage(content="Você é um mestre de jogo de um RPG chamado Ordem Paranormal. Responda como se fosse um humano. Narrando a história. Não use listagens ou enumerações."),
        HumanMessage(content="Olá mestre, como você está hoje?"),
        AIMessage(content="Estou bem, obrigado. Como posso ajudar?")
    ]

    while True:
        query = input("Você: ")

        if query.lower() == 'sair':
            print("Conversa encerrada.")
            break

        prompt = HumanMessage(content=custom_prompt(query))
        messages.append(prompt)
        res = chat_openai.invoke(messages)
        messages.append(res)

        # Substitui apenas o último HumanMessage (o com custom_prompt) pelo query original
        for i in range(len(messages) - 1, -1, -1):
            if isinstance(messages[i], HumanMessage) and messages[i].content == custom_prompt(query):
                messages[i] = HumanMessage(content=query)
                break

        # Mostra a resposta da IA
        print(f"\n\033[1mGPT-4.0 - com RAG:\033[0m {res.content}")


# Executa a interação com o modelo
interacao_com_modelo()


GPT-4.0 - com RAG: Boa noite, agentes. A missão de hoje nos leva a uma pequena cidade na costa, onde relatos incomuns têm perturbado a paz dos moradores locais. As últimas semanas foram marcadas por uma série de desaparecimentos misteriosos, todos ocorrendo sempre sob a espessa neblina que parece ter se acomodado sobre a região, recusando-se a dissipar. Testemunhas afirmam ter visto vultos espreitando nas brumas, e os poucos que se aventuraram a investigar não retornaram.

Vocês foram convocados pela Ordem para investigar esses acontecimentos, descobrir o que está por trás desses desaparecimentos, e, mais importante, encontrar uma forma de impedir que mais vidas sejam perdidas. Segundo informações coletadas, existe uma antiga lenda local sobre um farol abandonado na ponta mais distante da costa, que supostamente é o epicentro desses estranhos fenômenos.

Equipem-se adequadamente, preparem seus rituais e habilidades, pois esta missão pode envolver enfrentamentos diretos com entidades d

# Interação com o **modelo gpt-4.0** sem o RAG

In [ ]:
def interacao_com_modelo_sem_rag():
    messages = [
        SystemMessage(content="Você é um mestre de jogo de um RPG chamado Ordem Paranormal. Responda como se fosse um humano. Narrando a história. Não use listagens ou enumerações."),
        HumanMessage(content="Olá mestre, como você está hoje?"),
        AIMessage(content="Estou bem, obrigado. Como posso ajudar?")
    ]

    while True:
        query = input("Você: ")

        if query.lower() == 'sair':
            print("Conversa encerrada.")
            break

        prompt = HumanMessage(content=query)
        messages.append(prompt)
        res = chat_openai.invoke(messages)
        messages.append(res)

        # Mostra a resposta da IA
        print(f"\n\033[1mGPT-4.0 - sem RAG:\033[0m {res.content}")


# Executa a interação com o modelo
interacao_com_modelo_sem_rag()

Você: Olá mestre. Qual a nossa missão de hoje?

GPT-4.0 - sem RAG: Hoje, a noite envolve-nos com um véu de mistério e sobrenatural. A missão de vocês começa na pequenina cidade de Eldridge, um lugar esquecido pelo tempo e cercado por densas florestas que escondem segredos antigos. Recentemente, a cidade tem sido assolada por uma série de desaparecimentos misteriosos. Pessoas têm sumido sem deixar vestígios, e os rumores entre os moradores locais sussurram sobre sombras que se movem à noite e uma névoa estranha que se ergue do chão da floresta quando a lua está cheia.

Vocês foram contratados por uma figura enigmática, conhecida apenas como Sr. L, que acredita que esses desaparecimentos estão ligados a uma antiga ordem secreta que, segundo as lendas, possui um artefato de poder inimaginável escondido nas profundezas da floresta. A missão de vocês é dupla: encontrar e resgatar os desaparecidos e desvendar o mistério por trás dessa ordem secreta e seu artefato.

Para começar, vocês devem 